In [1]:
!pip install uaibot

In [3]:
import uaibot as ub
import numpy as np


def circulo():

  dt = 0.01
  t = 0
  tmax = 12


  def get_configuration(robot):
    return robot.q


  def set_configuration_speed(robot, qdot_des):
    q_next = robot.q + qdot_des * dt
    robot.add_ani_frame(time=t + dt, q=q_next)


  # As inicializações (ex: parâmetros do controlador) virão aqui
  robot = ub.Robot.create_kuka_kr5(color = "green")
  board_position = ub.Utils.trn([0, 0.77, 0.6]) #Create HTM for positioning the cube
  board = ub.Box(htm=board_position, width=0.75, depth=0.02, height=0.4, color="white") #Instantiate cube

  # Especifica a posição desejada para o efetuador
  #como uma função de t
  #Também fornece a derivada, que vai ser utilizada no feedforward
  R = 0.1
  y_c = 0.75
  z_c = 0.6
  omega_d = np.pi/2
  s_d = lambda tt: np.matrix([R * np.cos(omega_d * tt), y_c, z_c + R * np.sin(omega_d * tt)]).reshape((3, 1))
  s_d_dot = lambda tt: np.matrix([-R * omega_d * np.sin(omega_d * tt), 0, R * omega_d * np.cos(omega_d * tt)]).reshape((3, 1))


  # Especifica a orientação desejada para o eixo z do efetuador
  z_d = np.matrix([0,1,0]).reshape((3,1))

  # Cria uma nuvem de pontos para visualizarmos a parte de posição da referência
  pc = np.matrix(np.zeros((3,0)))
  for i in range(200):
    pc = np.block([pc, s_d(2*np.pi*i/199)])

  target_pos_pc = ub.PointCloud(points=pc, size=0.01, color="magenta")

  # Cria a simulação
  sim = ub.Simulation([robot, target_pos_pc, board],background_color="black")

  # Captura o número de juntas do robô
  n = np.shape(robot.q)[0]

  # Cria a função F:
  def fun_F(r):
      A = 0.25
      w_tol = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
      F = np.matrix(np.zeros((4, 1)))
      for i in range(4):
          if abs(r[i, 0]) < w_tol[i]:
              F[i, 0] = -A * (r[i, 0] / w_tol[i])
          elif r[i, 0] >= w_tol[i]:
              F[i, 0] = -A
          else:
              F[i, 0] = A
      return F

  # Cria uma matriz para o histórico de função de tarefa, da ação de controle
  # e do tempo
  hist_r = np.matrix(np.zeros((4, 0)))
  hist_u = np.matrix(np.zeros((n, 0)))
  hist_q = np.matrix(np.zeros((n,0)))
  hist_t = []

  # Colocaremos aqui nosso "main" do controlador, que ficará em um laço
  # durante um tempo tmax
  for i in range(round(tmax / dt)):
    #################################
    # Início da lógica de controle  #
    #################################

    # Mede a configuração dos sensores
    q = get_configuration(robot)

    # Calcula a cinemática direta e Jacobiana para o efetuador nessa
    # configuração
    Jg, fk = robot.jac_geo(q)
    # Faz a extração de z_e e s_e
    z_e = fk[0:3, 2]
    s_e = fk[0:3, 3]

    # Monta o vetor de tarefa
    r = np.matrix(np.zeros((4, 1)))
    r[0:3] = s_e - s_d(t)
    r[3] = 1 - z_d.T * z_e

    # Monta a Jacobiana de tarefa
    Jr = np.matrix(np.zeros((4, n)))

    # Monta o termo de feedforward
    ff = np.block([[-s_d_dot(t)],[0]])

    Jr[0:3, :] = Jg[0:3, :]
    Jr[3, :] = z_d.T * ub.Utils.S(z_e) * Jg[3:6, :]


    # Calcula a ação de controle
    u = ub.Utils.dp_inv(Jr, 0.001) * (fun_F(r)-ff)

    # Guarda informações no histórico
    hist_q = np.block([hist_q, q/(np.pi/180)])
    hist_r = np.block([hist_r, r])
    hist_u = np.block([hist_u, u*(180/np.pi)])
    hist_t.append(t)

    # Manda a ação de controle para o robô
    set_configuration_speed(robot, u)

    #################################
    # Fim da lógica de controle     #
    #################################

    # O tempo sempre vai passar no final do ciclo
    t += dt

  #Plota os graficos de configuração e ação de controle
  ub.Utils.plot(hist_t, hist_q, "", "Tempo (s)", "Configuração de junta (°)", "q")
  # ub.Utils.plot(hist_t, hist_r, "", "Tempo (s)", "Função de tarefa", "r")
  ub.Utils.plot(hist_t, hist_u, "", "Tempo (s)", "Ação (°/s)", "u")

  return sim


def triangulo():

  def get_configuration(robot):
    return robot.q

  def set_configuration_speed(robot, qdot_des):
    q_next = robot.q + qdot_des*dt
    robot.add_ani_frame(time = t+dt, q = q_next)

  dt = 0.01
  t = 0
  tmax = 6

  # As inicializações (ex: parâmetros do controlador) virão aqui
  robot = ub.Robot.create_kuka_kr5(color = "green")
  board_position = ub.Utils.trn([0, 0.77, 0.6]) #Create HTM for positioning the cube
  board = ub.Box(htm=board_position, width=0.75, depth=0.02, height=0.4, color="white") #Instantiate cube
  pc_traj = np.zeros((3, 0))

  #Especifica a pose 1 desejada para o efetuador
  htm_d = ub.Utils.trn([-0.1,0.2,-0.3]) * robot.fkm()
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d = htm_d[0:3,0]
  y_d = htm_d[0:3,1]
  z_d = htm_d[0:3,2]
  s_d = htm_d[0:3,3]

  #Especifica a pose 2 desejada para o efetuador
  htm_d2 = ub.Utils.trn([0.2,0,0]) * htm_d
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d2 = htm_d2[0:3,0]
  y_d2 = htm_d2[0:3,1]
  z_d2 = htm_d2[0:3,2]
  s_d2 = htm_d2[0:3,3]

  #Especifica a pose 3 desejada para o efetuador
  htm_d3 = ub.Utils.trn([-0.2,0,0.2]) * htm_d2
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d3 = htm_d3[0:3,0]
  y_d3 = htm_d3[0:3,1]
  z_d3 = htm_d3[0:3,2]
  s_d3 = htm_d3[0:3,3]



  #Cria um referencial no cenário para que seja possível ver se ele alcancou a
  #pose
  frame_d = ub.Frame(htm = htm_d)
  frame_d2 = ub.Frame(htm = htm_d2)
  frame_d3 = ub.Frame(htm = htm_d3)

  htm_d4 = htm_d

  poses_d = [htm_d,htm_d2,htm_d3,htm_d4]

  # Inicializa o índice da pose desejada
  current_pose_index = 0
  pose_tolerance = 0.01  # Limite de erro para considerar que o efetuador chegou à pose


  #Cria a simulação
  sim = ub.Simulation([robot,frame_d,frame_d2,frame_d3,board],background_color="black")

  #Captura o número de juntas do robô
  n = np.shape(robot.q)[0]

  #Cria a função F:
  def fun_F(r):
      A = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
      w_tol = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
      F = np.matrix(np.zeros((6, 1)))
      for i in range(6):
          if abs(r[i, 0]) < w_tol[i]:
              F[i, 0] = -A[i] * (r[i, 0] / w_tol[i])
          elif r[i, 0] >= w_tol[i]:
              F[i, 0] = -A[i]
          else:
              F[i, 0] = A[i]
      return F

  #Cria uma matriz para o histórico de função de tarefa, da ação de controle
  #e do tempo
  hist_r = np.matrix(np.zeros((6,0)))
  hist_u = np.matrix(np.zeros((n,0)))
  hist_q = np.matrix(np.zeros((n,0)))
  hist_t = []

  #Colocaremos aqui nosso "main" do controlador, que ficará em um laço
  #durante um tempo tmax

  for i in range(round(tmax/dt)):
      #################################
      # Início da lógica de controle  #
      #################################

      # Mede a configuração dos sensores
      q = get_configuration(robot)

      # Calcula a cinemática direta e Jacobiana para o efetuador nessa configuração
      Jg, fk = robot.jac_geo(q)

      # Extrai elementos da pose atual do efetuador
      x_e = fk[0:3, 0]
      y_e = fk[0:3, 1]
      z_e = fk[0:3, 2]
      s_e = fk[0:3, 3]

      # Atualiza a pose alvo atual
      htm_d_current = poses_d[current_pose_index]
      x_d = htm_d_current[0:3, 0]
      y_d = htm_d_current[0:3, 1]
      z_d = htm_d_current[0:3, 2]
      s_d = htm_d_current[0:3, 3]

      # Monta o vetor de tarefa
      r = np.matrix(np.zeros((6, 1)))
      r[0:3] = s_e - s_d
      r[3] = 1 - x_d.T * x_e
      r[4] = 1 - y_d.T * y_e
      r[5] = 1 - z_d.T * z_e

      # Monta a Jacobiana de tarefa
      Jr = np.matrix(np.zeros((6, n)))
      Jr[0:3, :] = Jg[0:3, :]
      Jr[3, :] = x_d.T * ub.Utils.S(x_e) * Jg[3:6, :]
      Jr[4, :] = y_d.T * ub.Utils.S(y_e) * Jg[3:6, :]
      Jr[5, :] = z_d.T * ub.Utils.S(z_e) * Jg[3:6, :]

      # Calcula a ação de controle
      u = ub.Utils.dp_inv(Jr, 0.001) * fun_F(r)

      # Guarda informações no histórico
      hist_q = np.block([hist_q, q/(np.pi/180)])
      hist_r = np.block([hist_r, r])
      hist_u = np.block([hist_u, u*(180/np.pi)])
      hist_t.append(t)

      # Manda a ação de controle para o robô
      set_configuration_speed(robot, u)

      # Atualiza a trajetória
      if current_pose_index > 0:  # Verifica se há mais poses
          pc_traj = np.block([pc_traj, s_e.reshape(3, 1)])

      # Verifica se o erro está abaixo do limite e troca para a próxima pose
      if np.linalg.norm(r) < pose_tolerance:
          if current_pose_index < len(poses_d) - 1:  # Verifica se há mais poses
              current_pose_index += 1
          else:
              break  # Finaliza o loop se todas as poses foram atingidas

      #################################
      # Fim da lógica de controle     #
      #################################

      # O tempo sempre vai passar no final do ciclo
      t += dt

  traj_points = ub.PointCloud(points=pc_traj, size=0.01, color="magenta")
  sim = ub.Simulation([robot,traj_points,board],background_color="black")

  #Plota os graficos de configuração e ação de controle
  ub.Utils.plot(hist_t, hist_q, "", "Tempo (s)", "Configuração de junta (°)", "q")
  # ub.Utils.plot(hist_t, hist_r, "", "Tempo (s)", "Função de tarefa", "r")
  ub.Utils.plot(hist_t, hist_u, "", "Tempo (s)", "Ação (°/s)", "u")

  return sim

def quadrado():
  dt = 0.01
  t = 0
  tmax = 6

  def get_configuration(robot):
    return robot.q

  def set_configuration_speed(robot, qdot_des):
    q_next = robot.q + qdot_des*dt
    robot.add_ani_frame(time = t+dt, q = q_next)


  #As inicializações (ex: parâmetros do controlador) virão aqui
  robot = ub.Robot.create_kuka_kr5(color="green")
  board_position = ub.Utils.trn([0, 0.77, 0.6]) #Create HTM for positioning the cube
  board = ub.Box(htm=board_position, width=0.75, depth=0.02, height=0.4, color="white") #Instantiate cube

  #Desenho da trajetória
  pc_traj = np.zeros((3, 0))

  #Especifica a pose 1 desejada para o efetuador
  htm_d = ub.Utils.trn([-0.1,0.2,-0.3]) * robot.fkm()
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d = htm_d[0:3,0]
  y_d = htm_d[0:3,1]
  z_d = htm_d[0:3,2]
  s_d = htm_d[0:3,3]

  #Especifica a pose 2 desejada para o efetuador
  htm_d2 = ub.Utils.trn([0.2,0,0]) * htm_d
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d2 = htm_d2[0:3,0]
  y_d2 = htm_d2[0:3,1]
  z_d2 = htm_d2[0:3,2]
  s_d2 = htm_d2[0:3,3]

  #Especifica a pose 3 desejada para o efetuador
  htm_d3 = ub.Utils.trn([0,0,0.2]) * htm_d2
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d3 = htm_d3[0:3,0]
  y_d3 = htm_d3[0:3,1]
  z_d3 = htm_d3[0:3,2]
  s_d3 = htm_d3[0:3,3]

  #Especifica a pose 3 desejada para o efetuador
  htm_d4 = ub.Utils.trn([-0.2,0,0]) * htm_d3
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d4 = htm_d4[0:3,0]
  y_d4 = htm_d4[0:3,1]
  z_d4 = htm_d4[0:3,2]
  s_d4 = htm_d4[0:3,3]

  #Cria um referencial no cenário para que seja possível ver se ele alcancou a
  #pose
  frame_d = ub.Frame(htm = htm_d)
  frame_d2 = ub.Frame(htm = htm_d2)
  frame_d3 = ub.Frame(htm = htm_d3)
  frame_d4 = ub.Frame(htm = htm_d4)

  htm_d5 = htm_d

  poses_d = [htm_d, htm_d2, htm_d3, htm_d4, htm_d5]

  # Inicializa o índice da pose desejada
  current_pose_index = 0
  pose_tolerance = 0.01  # Limite de erro para considerar que o efetuador chegou à pose


  #Cria a simulação
  sim = ub.Simulation([robot,frame_d,frame_d2,frame_d3,frame_d4,board],background_color="black")

  #Captura o número de juntas do robô
  n = np.shape(robot.q)[0]

  #Cria a função F:
  def fun_F(r):
      A = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
      w_tol = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
      F = np.matrix(np.zeros((6, 1)))
      for i in range(6):
          if abs(r[i, 0]) < w_tol[i]:
              F[i, 0] = -A[i] * (r[i, 0] / w_tol[i])
          elif r[i, 0] >= w_tol[i]:
              F[i, 0] = -A[i]
          else:
              F[i, 0] = A[i]
      return F

  #Cria uma matriz para o histórico de função de tarefa, da ação de controle
  #e do tempo
  hist_r = np.matrix(np.zeros((6,0)))
  hist_u = np.matrix(np.zeros((n,0)))
  hist_q = np.matrix(np.zeros((n,0)))
  hist_t = []

  #Colocaremos aqui nosso "main" do controlador, que ficará em um laço
  #durante um tempo tmax

  for i in range(round(tmax/dt)):
      #################################
      # Início da lógica de controle  #
      #################################

      # Mede a configuração dos sensores
      q = get_configuration(robot)

      # Calcula a cinemática direta e Jacobiana para o efetuador nessa configuração
      Jg, fk = robot.jac_geo(q)

      # Extrai elementos da pose atual do efetuador
      x_e = fk[0:3, 0]
      y_e = fk[0:3, 1]
      z_e = fk[0:3, 2]
      s_e = fk[0:3, 3]

      # Atualiza a pose alvo atual
      htm_d_current = poses_d[current_pose_index]
      x_d = htm_d_current[0:3, 0]
      y_d = htm_d_current[0:3, 1]
      z_d = htm_d_current[0:3, 2]
      s_d = htm_d_current[0:3, 3]

      # Monta o vetor de tarefa
      r = np.matrix(np.zeros((6, 1)))
      r[0:3] = s_e - s_d
      r[3] = 1 - x_d.T * x_e
      r[4] = 1 - y_d.T * y_e
      r[5] = 1 - z_d.T * z_e

      # Monta a Jacobiana de tarefa
      Jr = np.matrix(np.zeros((6, n)))
      Jr[0:3, :] = Jg[0:3, :]
      Jr[3, :] = x_d.T * ub.Utils.S(x_e) * Jg[3:6, :]
      Jr[4, :] = y_d.T * ub.Utils.S(y_e) * Jg[3:6, :]
      Jr[5, :] = z_d.T * ub.Utils.S(z_e) * Jg[3:6, :]

      # Calcula a ação de controle
      u = ub.Utils.dp_inv(Jr, 0.001) * fun_F(r)

      # Guarda informações no histórico
      hist_q = np.block([hist_q, q/(np.pi/180)])
      hist_r = np.block([hist_r, r])
      hist_u = np.block([hist_u, u*(180/np.pi)])
      hist_t.append(t)

      # Manda a ação de controle para o robô
      set_configuration_speed(robot, u)

      # Atualiza a trajetória
      if current_pose_index > 0:  # Verifica se há mais poses
          pc_traj = np.block([pc_traj, s_e.reshape(3, 1)])

      # Verifica se o erro está abaixo do limite e troca para a próxima pose
      if np.linalg.norm(r) < pose_tolerance:
          if current_pose_index < len(poses_d) - 1:  # Verifica se há mais poses
              current_pose_index += 1
          else:
              break  # Finaliza o loop se todas as poses foram atingidas

      #################################
      # Fim da lógica de controle     #
      #################################

      # O tempo sempre vai passar no final do ciclo
      t += dt

  traj_points = ub.PointCloud(points=pc_traj, size=0.01, color="magenta")

  sim = ub.Simulation([robot,board,traj_points],background_color="black")

  #Plota os graficos de configuração e ação de controle
  ub.Utils.plot(hist_t, hist_q, "", "Tempo (s)", "Configuração de junta (°)", "q")
  # ub.Utils.plot(hist_t, hist_r, "", "Tempo (s)", "Função de tarefa", "r")
  ub.Utils.plot(hist_t, hist_u, "", "Tempo (s)", "Ação (°/s)", "u")

  return sim

def trapezio():
  dt = 0.01
  t = 0
  tmax = 6

  def get_configuration(robot):
    return robot.q

  def set_configuration_speed(robot, qdot_des):
    q_next = robot.q + qdot_des*dt
    robot.add_ani_frame(time = t+dt, q = q_next)


  #As inicializações (ex: parâmetros do controlador) virão aqui
  robot = ub.Robot.create_kuka_kr5(color="green")
  board_position = ub.Utils.trn([0, 0.78, 0.6]) #Create HTM for positioning the cube
  board = ub.Box(htm=board_position, width=0.75, depth=0.02, height=0.4, color="white") #Instantiate cube
  pc_traj = np.zeros((3, 0))

  #Especifica a pose 1 desejada para o efetuador
  htm_d = ub.Utils.trn([-0.1,0.2,-0.3]) * robot.fkm()
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d = htm_d[0:3,0]
  y_d = htm_d[0:3,1]
  z_d = htm_d[0:3,2]
  s_d = htm_d[0:3,3]

  #Especifica a pose 2 desejada para o efetuador
  htm_d2 = ub.Utils.trn([0.2,0,0.1]) * htm_d
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d2 = htm_d2[0:3,0]
  y_d2 = htm_d2[0:3,1]
  z_d2 = htm_d2[0:3,2]
  s_d2 = htm_d2[0:3,3]

  #Especifica a pose 3 desejada para o efetuador
  htm_d3 = ub.Utils.trn([0.1,0,-0.1]) * htm_d2
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d3 = htm_d3[0:3,0]
  y_d3 = htm_d3[0:3,1]
  z_d3 = htm_d3[0:3,2]
  s_d3 = htm_d3[0:3,3]

  #Especifica a pose 4 desejada para o efetuador
  htm_d4 = ub.Utils.trn([0.2,0,-0.2]) * htm_d3
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d4 = htm_d4[0:3,0]
  y_d4 = htm_d4[0:3,1]
  z_d4 = htm_d4[0:3,2]
  s_d4 = htm_d4[0:3,3]



  #Cria um referencial no cenário para que seja possível ver se ele alcancou a
  #pose
  frame_d = ub.Frame(htm = htm_d)
  frame_d2 = ub.Frame(htm = htm_d2)
  frame_d3 = ub.Frame(htm = htm_d3)
  frame_d4 = ub.Frame(htm = htm_d4)
  frame_d5 = ub.Frame(htm = htm_d)

  htm_d5 = htm_d

  poses_d = [htm_d,htm_d2,htm_d3,htm_d5]
  #poses_d = [htm_d,htm_d2,htm_d3,htm_d4,htm_d5]

  # Inicializa o índice da pose desejada
  current_pose_index = 0
  pose_tolerance = 0.01  # Limite de erro para considerar que o efetuador chegou à pose


  #Cria a simulação
  sim = ub.Simulation([robot,frame_d,frame_d2,frame_d3,frame_d4,board],background_color="black")

  #Captura o número de juntas do robô
  n = np.shape(robot.q)[0]

  #Cria a função F:
  def fun_F(r):
      A = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
      w_tol = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
      F = np.matrix(np.zeros((6, 1)))
      for i in range(6):
          if abs(r[i, 0]) < w_tol[i]:
              F[i, 0] = -A[i] * (r[i, 0] / w_tol[i])
          elif r[i, 0] >= w_tol[i]:
              F[i, 0] = -A[i]
          else:
              F[i, 0] = A[i]
      return F

  #Cria uma matriz para o histórico de função de tarefa, da ação de controle
  #e do tempo
  hist_r = np.matrix(np.zeros((6,0)))
  hist_u = np.matrix(np.zeros((n,0)))
  hist_q = np.matrix(np.zeros((n,0)))
  hist_t = []

  #Colocaremos aqui nosso "main" do controlador, que ficará em um laço
  #durante um tempo tmax

  for i in range(round(tmax/dt)):
      #################################
      # Início da lógica de controle  #
      #################################

      # Mede a configuração dos sensores
      q = get_configuration(robot)

      # Calcula a cinemática direta e Jacobiana para o efetuador nessa configuração
      Jg, fk = robot.jac_geo(q)

      # Extrai elementos da pose atual do efetuador
      x_e = fk[0:3, 0]
      y_e = fk[0:3, 1]
      z_e = fk[0:3, 2]
      s_e = fk[0:3, 3]

      # Atualiza a pose alvo atual
      htm_d_current = poses_d[current_pose_index]
      x_d = htm_d_current[0:3, 0]
      y_d = htm_d_current[0:3, 1]
      z_d = htm_d_current[0:3, 2]
      s_d = htm_d_current[0:3, 3]

      # Monta o vetor de tarefa
      r = np.matrix(np.zeros((6, 1)))
      r[0:3] = s_e - s_d
      r[3] = 1 - x_d.T * x_e
      r[4] = 1 - y_d.T * y_e
      r[5] = 1 - z_d.T * z_e

      # Monta a Jacobiana de tarefa
      Jr = np.matrix(np.zeros((6, n)))
      Jr[0:3, :] = Jg[0:3, :]
      Jr[3, :] = x_d.T * ub.Utils.S(x_e) * Jg[3:6, :]
      Jr[4, :] = y_d.T * ub.Utils.S(y_e) * Jg[3:6, :]
      Jr[5, :] = z_d.T * ub.Utils.S(z_e) * Jg[3:6, :]

      # Calcula a ação de controle
      u = ub.Utils.dp_inv(Jr, 0.001) * fun_F(r)

      # Guarda informações no histórico
      hist_q = np.block([hist_q, q/(np.pi/180)])
      hist_r = np.block([hist_r, r])
      hist_u = np.block([hist_u, u*(180/np.pi)])
      hist_t.append(t)

      # Manda a ação de controle para o robô
      set_configuration_speed(robot, u)

      # Atualiza a trajetória
      if current_pose_index > 0:  # Verifica se há mais poses
          pc_traj = np.block([pc_traj, s_e.reshape(3, 1)])

      # Verifica se o erro está abaixo do limite e troca para a próxima pose
      if np.linalg.norm(r) < pose_tolerance:
          if current_pose_index < len(poses_d) - 1:  # Verifica se há mais poses
              current_pose_index += 1
          else:
              break  # Finaliza o loop se todas as poses foram atingidas

      #################################
      # Fim da lógica de controle     #
      #################################

      # O tempo sempre vai passar no final do ciclo
      t += dt

  traj_points = ub.PointCloud(points=pc_traj, size=0.01, color="magenta")
  sim = ub.Simulation([robot,traj_points,board],background_color="black")

  #Plota os graficos de configuração e ação de controle
  ub.Utils.plot(hist_t, hist_q, "", "Tempo (s)", "Configuração de junta (°)", "q")
  # ub.Utils.plot(hist_t, hist_r, "", "Tempo (s)", "Função de tarefa", "r")
  ub.Utils.plot(hist_t, hist_u, "", "Tempo (s)", "Ação (°/s)", "u")

  return sim

def hexagono():
  dt = 0.01
  t = 0
  tmax = 6

  def get_configuration(robot):
    return robot.q

  def set_configuration_speed(robot, qdot_des):
    q_next = robot.q + qdot_des*dt
    robot.add_ani_frame(time = t+dt, q = q_next)


  #As inicializações (ex: parâmetros do controlador) virão aqui
  robot = ub.Robot.create_kuka_kr5(color="green")
  board_position = ub.Utils.trn([0, 0.78, 0.6]) #Create HTM for positioning the cube
  board = ub.Box(htm=board_position, width=0.75, depth=0.02, height=0.4, color="white") #Instantiate cube
  pc_traj = np.zeros((3, 0))

  #Especifica a pose 1 desejada para o efetuador
  htm_d = ub.Utils.trn([-0.1,0.2,-0.3]) * robot.fkm()
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d = htm_d[0:3,0]
  y_d = htm_d[0:3,1]
  z_d = htm_d[0:3,2]
  s_d = htm_d[0:3,3]

  #Especifica a pose 2 desejada para o efetuador
  htm_d2 = ub.Utils.trn([0.15,0,0]) * htm_d
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d2 = htm_d2[0:3,0]
  y_d2 = htm_d2[0:3,1]
  z_d2 = htm_d2[0:3,2]
  s_d2 = htm_d2[0:3,3]

  #Especifica a pose 3 desejada para o efetuador
  htm_d3 = ub.Utils.trn([0.1,0,0.1]) * htm_d2
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d3 = htm_d3[0:3,0]
  y_d3 = htm_d3[0:3,1]
  z_d3 = htm_d3[0:3,2]
  s_d3 = htm_d3[0:3,3]

  #Especifica a pose 4 desejada para o efetuador
  htm_d4 = ub.Utils.trn([-0.1,0,0.1]) * htm_d3
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d4 = htm_d4[0:3,0]
  y_d4 = htm_d4[0:3,1]
  z_d4 = htm_d4[0:3,2]
  s_d4 = htm_d4[0:3,3]

  #Especifica a pose 5 desejada para o efetuador
  htm_d5 = ub.Utils.trn([-0.15,0,0]) * htm_d4
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d5 = htm_d5[0:3,0]
  y_d5 = htm_d5[0:3,1]
  z_d5 = htm_d5[0:3,2]
  s_d5 = htm_d5[0:3,3]

  #Especifica a pose 6 desejada para o efetuador
  htm_d6 = ub.Utils.trn([-0.1,0,-0.1]) * htm_d5
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d6 = htm_d6[0:3,0]
  y_d6 = htm_d6[0:3,1]
  z_d6 = htm_d6[0:3,2]
  s_d6 = htm_d6[0:3,3]



  #Cria um referencial no cenário para que seja possível ver se ele alcancou a
  #pose
  frame_d = ub.Frame(htm = htm_d)
  frame_d2 = ub.Frame(htm = htm_d2)
  frame_d3 = ub.Frame(htm = htm_d3)
  frame_d4 = ub.Frame(htm = htm_d4)
  frame_d5 = ub.Frame(htm = htm_d5)
  frame_d6 = ub.Frame(htm = htm_d6)

  htm_d7 = htm_d

  poses_d = [htm_d,htm_d2,htm_d3,htm_d4,htm_d5,htm_d6,htm_d7]

  # Inicializa o índice da pose desejada
  current_pose_index = 0
  pose_tolerance = 0.01  # Limite de erro para considerar que o efetuador chegou à pose


  #Cria a simulação
  sim = ub.Simulation([robot,frame_d,frame_d2,frame_d3,frame_d4,board],background_color="black")

  #Captura o número de juntas do robô
  n = np.shape(robot.q)[0]

  #Cria a função F:
  def fun_F(r):
      A = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
      w_tol = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
      F = np.matrix(np.zeros((6, 1)))
      for i in range(6):
          if abs(r[i, 0]) < w_tol[i]:
              F[i, 0] = -A[i] * (r[i, 0] / w_tol[i])
          elif r[i, 0] >= w_tol[i]:
              F[i, 0] = -A[i]
          else:
              F[i, 0] = A[i]
      return F

  #Cria uma matriz para o histórico de função de tarefa, da ação de controle
  #e do tempo
  hist_r = np.matrix(np.zeros((6,0)))
  hist_u = np.matrix(np.zeros((n,0)))
  hist_q = np.matrix(np.zeros((n,0)))
  hist_t = []

  #Colocaremos aqui nosso "main" do controlador, que ficará em um laço
  #durante um tempo tmax

  for i in range(round(tmax/dt)):
      #################################
      # Início da lógica de controle  #
      #################################

      # Mede a configuração dos sensores
      q = get_configuration(robot)

      # Calcula a cinemática direta e Jacobiana para o efetuador nessa configuração
      Jg, fk = robot.jac_geo(q)

      # Extrai elementos da pose atual do efetuador
      x_e = fk[0:3, 0]
      y_e = fk[0:3, 1]
      z_e = fk[0:3, 2]
      s_e = fk[0:3, 3]

      # Atualiza a pose alvo atual
      htm_d_current = poses_d[current_pose_index]
      x_d = htm_d_current[0:3, 0]
      y_d = htm_d_current[0:3, 1]
      z_d = htm_d_current[0:3, 2]
      s_d = htm_d_current[0:3, 3]

      # Monta o vetor de tarefa
      r = np.matrix(np.zeros((6, 1)))
      r[0:3] = s_e - s_d
      r[3] = 1 - x_d.T * x_e
      r[4] = 1 - y_d.T * y_e
      r[5] = 1 - z_d.T * z_e

      # Monta a Jacobiana de tarefa
      Jr = np.matrix(np.zeros((6, n)))
      Jr[0:3, :] = Jg[0:3, :]
      Jr[3, :] = x_d.T * ub.Utils.S(x_e) * Jg[3:6, :]
      Jr[4, :] = y_d.T * ub.Utils.S(y_e) * Jg[3:6, :]
      Jr[5, :] = z_d.T * ub.Utils.S(z_e) * Jg[3:6, :]

      # Calcula a ação de controle
      u = ub.Utils.dp_inv(Jr, 0.001) * fun_F(r)

      # Guarda informações no histórico
      hist_q = np.block([hist_q, q/(np.pi/180)])
      hist_r = np.block([hist_r, r])
      hist_u = np.block([hist_u, u*(180/np.pi)])
      hist_t.append(t)

      # Manda a ação de controle para o robô
      set_configuration_speed(robot, u)

      # Atualiza a trajetória
      if current_pose_index > 0:  # Verifica se há mais poses
          pc_traj = np.block([pc_traj, s_e.reshape(3, 1)])

      # Verifica se o erro está abaixo do limite e troca para a próxima pose
      if np.linalg.norm(r) < pose_tolerance:
          if current_pose_index < len(poses_d) - 1:  # Verifica se há mais poses
              current_pose_index += 1
          else:
              break  # Finaliza o loop se todas as poses foram atingidas

      #################################
      # Fim da lógica de controle     #
      #################################

      # O tempo sempre vai passar no final do ciclo
      t += dt

  traj_points = ub.PointCloud(points=pc_traj, size=0.01, color="magenta")
  sim = ub.Simulation([robot,traj_points,board],background_color="black")

  #Plota os graficos de configuração e ação de controle
  ub.Utils.plot(hist_t, hist_q, "", "Tempo (s)", "Configuração de junta (°)", "q")
  # ub.Utils.plot(hist_t, hist_r, "", "Tempo (s)", "Função de tarefa", "r")
  ub.Utils.plot(hist_t, hist_u, "", "Tempo (s)", "Ação (°/s)", "u")

  return sim

def octogono():

  dt = 0.01
  t = 0
  tmax = 6

  def get_configuration(robot):
    return robot.q

  def set_configuration_speed(robot, qdot_des):
    q_next = robot.q + qdot_des*dt
    robot.add_ani_frame(time = t+dt, q = q_next)


  #As inicializações (ex: parâmetros do controlador) virão aqui
  robot = ub.Robot.create_kuka_kr5(color="green")
  board_position = ub.Utils.trn([0, 0.78, 0.6]) #Create HTM for positioning the cube
  board = ub.Box(htm=board_position, width=0.75, depth=0.02, height=0.4, color="white") #Instantiate cube
  pc_traj = np.zeros((3, 0))

  #Especifica a pose 1 desejada para o efetuador
  htm_d = ub.Utils.trn([-0.1,0.2,-0.3]) * robot.fkm()
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d = htm_d[0:3,0]
  y_d = htm_d[0:3,1]
  z_d = htm_d[0:3,2]
  s_d = htm_d[0:3,3]

  #Especifica a pose 2 desejada para o efetuador
  htm_d2 = ub.Utils.trn([0.1,0,0]) * htm_d
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d2 = htm_d2[0:3,0]
  y_d2 = htm_d2[0:3,1]
  z_d2 = htm_d2[0:3,2]
  s_d2 = htm_d2[0:3,3]

  #Especifica a pose 3 desejada para o efetuador
  htm_d3 = ub.Utils.trn([0.1,0,0.1]) * htm_d2
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d3 = htm_d3[0:3,0]
  y_d3 = htm_d3[0:3,1]
  z_d3 = htm_d3[0:3,2]
  s_d3 = htm_d3[0:3,3]

  #Especifica a pose 4 desejada para o efetuador
  htm_d4 = ub.Utils.trn([0,0,0.1]) * htm_d3
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d4 = htm_d4[0:3,0]
  y_d4 = htm_d4[0:3,1]
  z_d4 = htm_d4[0:3,2]
  s_d4 = htm_d4[0:3,3]

  #Especifica a pose 5 desejada para o efetuador
  htm_d5 = ub.Utils.trn([-0.1,0,0.1]) * htm_d4
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d5 = htm_d5[0:3,0]
  y_d5 = htm_d5[0:3,1]
  z_d5 = htm_d5[0:3,2]
  s_d5 = htm_d5[0:3,3]

  #Especifica a pose 6 desejada para o efetuador
  htm_d6 = ub.Utils.trn([-0.1,0,0]) * htm_d5
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d6 = htm_d6[0:3,0]
  y_d6 = htm_d6[0:3,1]
  z_d6 = htm_d6[0:3,2]
  s_d6 = htm_d6[0:3,3]

  #Especifica a pose 7 desejada para o efetuador
  htm_d7 = ub.Utils.trn([-0.1,0,-0.1]) * htm_d6
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d7 = htm_d7[0:3,0]
  y_d7 = htm_d7[0:3,1]
  z_d7 = htm_d7[0:3,2]
  s_d7 = htm_d7[0:3,3]

  #Especifica a pose 8 desejada para o efetuador
  htm_d8 = ub.Utils.trn([0,0,-0.1]) * htm_d7
  #Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d8 = htm_d8[0:3,0]
  y_d8 = htm_d8[0:3,1]
  z_d8 = htm_d8[0:3,2]
  s_d8 = htm_d8[0:3,3]



  #Cria um referencial no cenário para que seja possível ver se ele alcancou a
  #pose
  frame_d = ub.Frame(htm = htm_d)
  frame_d2 = ub.Frame(htm = htm_d2)
  frame_d3 = ub.Frame(htm = htm_d3)
  frame_d4 = ub.Frame(htm = htm_d4)
  frame_d5 = ub.Frame(htm = htm_d5)
  frame_d6 = ub.Frame(htm = htm_d6)
  frame_d7 = ub.Frame(htm = htm_d7)
  frame_d8 = ub.Frame(htm = htm_d8)

  htm_d9 = htm_d

  poses_d = [htm_d,htm_d2,htm_d3,htm_d4,htm_d5,htm_d6,htm_d7,htm_d8,htm_d9]

  # Inicializa o índice da pose desejada
  current_pose_index = 0
  pose_tolerance = 0.01  # Limite de erro para considerar que o efetuador chegou à pose


  #Cria a simulação
  sim = ub.Simulation([robot,frame_d,frame_d2,frame_d3,frame_d4,board],background_color="black")

  #Captura o número de juntas do robô
  n = np.shape(robot.q)[0]

  #Cria a função F:
  def fun_F(r):
      A = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
      w_tol = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
      F = np.matrix(np.zeros((6, 1)))
      for i in range(6):
          if abs(r[i, 0]) < w_tol[i]:
              F[i, 0] = -A[i] * (r[i, 0] / w_tol[i])
          elif r[i, 0] >= w_tol[i]:
              F[i, 0] = -A[i]
          else:
              F[i, 0] = A[i]
      return F

  #Cria uma matriz para o histórico de função de tarefa, da ação de controle
  #e do tempo
  hist_r = np.matrix(np.zeros((6,0)))
  hist_u = np.matrix(np.zeros((n,0)))
  hist_q = np.matrix(np.zeros((n,0)))
  hist_t = []

  #Colocaremos aqui nosso "main" do controlador, que ficará em um laço
  #durante um tempo tmax

  for i in range(round(tmax/dt)):
      #################################
      # Início da lógica de controle  #
      #################################

      # Mede a configuração dos sensores
      q = get_configuration(robot)

      # Calcula a cinemática direta e Jacobiana para o efetuador nessa configuração
      Jg, fk = robot.jac_geo(q)

      # Extrai elementos da pose atual do efetuador
      x_e = fk[0:3, 0]
      y_e = fk[0:3, 1]
      z_e = fk[0:3, 2]
      s_e = fk[0:3, 3]

      # Atualiza a pose alvo atual
      htm_d_current = poses_d[current_pose_index]
      x_d = htm_d_current[0:3, 0]
      y_d = htm_d_current[0:3, 1]
      z_d = htm_d_current[0:3, 2]
      s_d = htm_d_current[0:3, 3]

      # Monta o vetor de tarefa
      r = np.matrix(np.zeros((6, 1)))
      r[0:3] = s_e - s_d
      r[3] = 1 - x_d.T * x_e
      r[4] = 1 - y_d.T * y_e
      r[5] = 1 - z_d.T * z_e

      # Monta a Jacobiana de tarefa
      Jr = np.matrix(np.zeros((6, n)))
      Jr[0:3, :] = Jg[0:3, :]
      Jr[3, :] = x_d.T * ub.Utils.S(x_e) * Jg[3:6, :]
      Jr[4, :] = y_d.T * ub.Utils.S(y_e) * Jg[3:6, :]
      Jr[5, :] = z_d.T * ub.Utils.S(z_e) * Jg[3:6, :]

      # Calcula a ação de controle
      u = ub.Utils.dp_inv(Jr, 0.001) * fun_F(r)

      # Guarda informações no histórico
      hist_q = np.block([hist_q, q/(np.pi/180)])
      hist_r = np.block([hist_r, r])
      hist_u = np.block([hist_u, u*(180/np.pi)])
      hist_t.append(t)

      # Manda a ação de controle para o robô
      set_configuration_speed(robot, u)

      # Atualiza a trajetória
      if current_pose_index > 0:  # Verifica se há mais poses
          pc_traj = np.block([pc_traj, s_e.reshape(3, 1)])

      # Verifica se o erro está abaixo do limite e troca para a próxima pose
      if np.linalg.norm(r) < pose_tolerance:
          if current_pose_index < len(poses_d) - 1:  # Verifica se há mais poses
              current_pose_index += 1
          else:
              break  # Finaliza o loop se todas as poses foram atingidas

      #################################
      # Fim da lógica de controle     #
      #################################

      # O tempo sempre vai passar no final do ciclo
      t += dt

  traj_points = ub.PointCloud(points=pc_traj, size=0.01, color="magenta")
  sim = ub.Simulation([robot,traj_points,board],background_color="black")

  #Plota os graficos de configuração e ação de controle
  ub.Utils.plot(hist_t, hist_q, "", "Tempo (s)", "Configuração de junta (°)", "q")
  # ub.Utils.plot(hist_t, hist_r, "", "Tempo (s)", "Função de tarefa", "r")
  ub.Utils.plot(hist_t, hist_u, "", "Tempo (s)", "Ação (°/s)", "u")

  return sim





#Interface de usuário
print("Escolha a forma a ser desenhada: ")
print("1 - Circulo \n2 - Triângulo \n3 - Quadrado \n4 - Trapézio \n5 - Hexágono \n6 - Octógono")
selection = int(input('>'))

if selection == 1:
  sim = circulo()
elif selection == 2:
  sim = triangulo()
elif selection == 3:
  sim = quadrado()
elif selection == 4:
  sim = trapezio()
elif selection == 5:
  sim = hexagono()
elif selection == 6:
  sim = octogono()

#Roda a simulação
sim.run()

Escolha a forma a ser desenhada: 
1 - Circulo 
2 - Triângulo 
3 - Quadrado 
4 - Trapézio 
5 - Hexágono 
6 - Octógono
>5
